In [59]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [60]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

### Create our QandA application

In [61]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch   

In [62]:
file = "OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

In [63]:
embeddings = OpenAIEmbeddings()

In [64]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings).from_loaders([loader])

In [65]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={
        "document_separator": "<<<<<>>>>>"},
)

### Hard-coded examples

In [66]:
data[112]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 112}, page_content=": 112\nname: Boys'  Tech Quarter-Zip Pullover\ndescription: This pullover is perfect for any active lifestyle. It is moisture-wicking and quick-drying, while also being incredibly comfortable and stretchy. The fabric stretches with him, but won't stretch out of shape. It also features a flat-faced fleece fabric that is brushed on the inside so it feels great against his skin. Additionally, it has reflective hits for enhanced visibility in low-light conditions. \n\nSize & Fit: Relaxed Fit. \n\nFabric & Care: 88% polyester, 12% elastane. Machine wash and dry. \n\nImported.")

In [67]:
examples = [
    {
        "query":"Is the Tech Quarter-Zip Pullover a good choice for active lifestyle?",
        "answer":"Yes"
    },
    {
        "query":"What makes the Tech Quarter-Zip Pulliover feel greate against the skin?",
        "answer":"It features a flat-faced fleece fabric that is brushed on the inside."
    }
]

### LLM-Generated exmaples

In [68]:
from langchain.evaluation.qa import QAGenerateChain

In [71]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [74]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

new_examples = [{"query": ex["qa_pairs"]["query"], "answer": ex["qa_pairs"]["answer"]} for ex in new_examples]

c:\Users\trant\anaconda3\envs\fun\lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [75]:
new_examples[0]

{'query': "According to the document, what material are the Women's Campside Oxfords made of for a broken-in feel and look?",
 'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look."}

In [76]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [77]:
examples += new_examples
print(len(examples))
examples

7


[{'query': 'Is the Tech Quarter-Zip Pullover a good choice for active lifestyle?',
  'answer': 'Yes'},
 {'query': 'What makes the Tech Quarter-Zip Pulliover feel greate against the skin?',
  'answer': 'It features a flat-faced fleece fabric that is brushed on the inside.'},
 {'query': "According to the document, what material are the Women's Campside Oxfords made of for a broken-in feel and look?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look."},
 {'query': 'What are the dimensions of the small and medium sizes for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The dimensions of the small size is 18" x 28" and the dimensions of the medium size is 22.5" x 34.5".'},
 {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece as described in the document?",
  'answer': 'Some key features of the swimsuit include bright colors, ruffles, exclusive whimsical prints, four-way

In [78]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Tech Quarter-Zip Pullover is a good choice for an active lifestyle. It is moisture-wicking, quick-drying, comfortable, and stretchy, making it suitable for various activities. Additionally, it has reflective hits for enhanced visibility in low-light conditions.'

### Manual Evaluation

In [79]:
import langchain
langchain.debug = True

In [80]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Is the Tech Quarter-Zip Pullover a good choice for active lifestyle?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Is the Tech Quarter-Zip Pullover a good choice for active lifestyle?",
  "context": ": 112\nname: Boys'  Tech Quarter-Zip Pullover\ndescription: This pullover is perfect for any active lifestyle. It is moisture-wicking and quick-drying, while also being incredibly comfortable and stretchy. The fabric stretches with him, but won't stretch out of shape. It also features a flat-faced fleece fabric that is brushed on the inside so it feels great against his skin. Additionally, it has reflective hits for enhanced visibility in low-light conditions. \n\nSize & Fit: Relaxed Fit. \n\nFabric & Care: 88% polyester, 12% ela

'Yes, the Tech Quarter-Zip Pullover is a good choice for an active lifestyle. It is moisture-wicking, quick-drying, comfortable, and stretchy, making it suitable for various activities. Additionally, it has reflective hits for enhanced visibility in low-light conditions.'

In [81]:
langchain.debug = False

### LLM assisted evaluation

In [88]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [89]:
predictions

[{'query': 'Is the Tech Quarter-Zip Pullover a good choice for active lifestyle?',
  'answer': 'Yes',
  'result': 'Yes, the Tech Quarter-Zip Pullover is a good choice for an active lifestyle. It is moisture-wicking, quick-drying, comfortable, and stretchy, making it suitable for various activities. Additionally, it has reflective hits for enhanced visibility in low-light conditions.'},
 {'query': 'What makes the Tech Quarter-Zip Pulliover feel greate against the skin?',
  'answer': 'It features a flat-faced fleece fabric that is brushed on the inside.',
  'result': 'The Tech Quarter-Zip Pullover feels great against the skin because it features a flat-faced fleece fabric that is brushed on the inside.'},
 {'query': "According to the document, what material are the Women's Campside Oxfords made of for a broken-in feel and look?",
  'answer': "The Women's Campside Oxfords are made of soft canvas material for a broken-in feel and look.",
  'result': "The Women's Campside Oxfords are made o

In [90]:
from langchain.evaluation.qa import QAEvalChain

In [91]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [93]:
graded_output = eval_chain.evaluate(examples, predictions)
graded_output

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [95]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print(f"Question: {predictions[i]['query']}")
    print(f"Real Answer: {predictions[i]['answer']}")
    print(f"Predicted Answer: {predictions[i]['result']}")
    print(f"Predicted Grade: {graded_output[i]['results']}")
    print()

Example 0:
Question: Is the Tech Quarter-Zip Pullover a good choice for active lifestyle?
Real Answer: Yes
Predicted Answer: Yes, the Tech Quarter-Zip Pullover is a good choice for an active lifestyle. It is moisture-wicking, quick-drying, comfortable, and stretchy, making it suitable for various activities. Additionally, it has reflective hits for enhanced visibility in low-light conditions.
Predicted Grade: CORRECT

Example 1:
Question: What makes the Tech Quarter-Zip Pulliover feel greate against the skin?
Real Answer: It features a flat-faced fleece fabric that is brushed on the inside.
Predicted Answer: The Tech Quarter-Zip Pullover feels great against the skin because it features a flat-faced fleece fabric that is brushed on the inside.
Predicted Grade: CORRECT

Example 2:
Question: According to the document, what material are the Women's Campside Oxfords made of for a broken-in feel and look?
Real Answer: The Women's Campside Oxfords are made of soft canvas material for a broken